<a href="https://colab.research.google.com/github/jx-dohwan/Aiffel_Exp_Project/blob/main/fine_tuning/fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# fine-tuning

In [1]:

!pip install torch
!pip install transformers==4.25.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:

!git clone https://github.com/smilegate-ai/korean_smile_style_dataset
# https://corpus.korean.go.kr/
# https://aihub.or.kr/aihub-data/natural-language/about
     

fatal: destination path 'korean_smile_style_dataset' already exists and is not an empty directory.


In [3]:
import pandas as pd
df = pd.read_csv('/content/korean_smile_style_dataset/smilestyle_dataset.tsv', sep='\t')
df.to_csv('./korean_smile_style_dataset/smile.csv', index=False)

In [4]:
!mv /content/korean_smile_style_dataset/smile.csv /content/drive/MyDrive/인공지능/멀티턴응답선택/korean_smile_style_dataset

## 학습/검증 분할하기|

In [5]:
import csv
f = open('/content/drive/MyDrive/인공지능/멀티턴응답선택/korean_smile_style_dataset/smile.csv', 'r')
rdr = csv.reader(f)

""" 세션 데이터 저장할 것 """
session_dataset = []
session = []
all_utts = []

""" 실제 데이터 저장 방식 """
for i, line in enumerate(rdr):
    if i == 0:
        header = line
    else:
        utt = line[0]
        if utt.strip() != '':
            all_utts.append(utt)
            session.append(utt)
        else:
            """ 세션 데이터 저장 """
            session_dataset.append(session)
            session = []
""" 마지막 세션 저장 """
session_dataset.append(session)
f.close()

In [6]:
!head -20 /content/drive/MyDrive/인공지능/멀티턴응답선택/korean_smile_style_dataset/smile.csv

formal,informal,android,azae,chat,choding,emoticon,enfp,gentle,halbae,halmae,joongding,king,naruto,seonbi,sosim,translator
안녕하세요. 저는 고양이 6마리 키워요.,안녕! 나는 고양이 6마리 키워.,휴먼. 반갑다. 안드로이드는. 고양이. 6마리. 소유중.,아이고 안녕하십니까~ 나는 그냥 고양이 6마리 키우고 있는 사람이여,하잉ㅋㅋ 나 떼걸룩 6마리 키운다!,ㅎㅇ 나 주인님 6마리 모심 ㅋㅋ,안녕!! >< 나는 고양이😺를 ➏ 마리 키우고있어!! 0_0,안녕안녕~! 나 고양이 6마리나 키운다? 완전 대박이징~,"안녕하십니까,, 저는 고양이 6마리 키웁니다.",안녕하신가~... 난 지금 고양이를 6마리 키우고 있다네,하유 시벌것 괭이놈 6마리 키우는데 힘들어 죽겟네,안녕하냐 ㅡㅡ 나 씹냥이 6마리나 키운다 하;,반갑소. 짐은 고양이를 6마리나 키우오.,안녕하냐니깐! 난 고양이를 6마리 키우고있다니깐!,안녕하시오! 소인은 고양이를 6마리 키우고 있소!,안녕… 난 고양이 6마리 키워 ㅠㅠ,반가운. 나는 6마리의 고양이를 소지하고 있다.
고양이를 6마리나요? 키우는거 안 힘드세요?,고양이를 6마리나? 키우는거 안 힘들어?,고양이. 6마리. 양육. 번거로운가.,아니 무슨 고양이를 6마리나? 거 키우는 거 안 힘든가?,엥? 6마리나? 안힘듬?ㅋㅋㅋㅋ,6마리? 에바아니냐 안 힘듦?,고양이를 6마리나?!! w(ﾟДﾟ)w 키우는거 안 힘듬?? (⊙_⊙;),고양이를 6마리나? 완전 대박~ 키우는 거 안 힘들어?!,"고양이를 6마리나 키우십니까? 안 힘드신지,,",고양이를 6마리나? 키우는거 힘들지 않는가?,니기럴 털만 날리는 거 키우기 안 힘들데?,아니 고양이를 6마리나? 안힘드냐?,고양이를 6마리나? 키우는게 수고스럽진 않소?,고양이를 6마리나? 키우는거 힘들지 않냐니깐?,고양이를 6마리나 키우고 있는 것이오? 힘들지 않소?,고양이..6마리나? ㅠ 키우는건 혹시 안힘들어..?,6마리의

In [7]:
print(len(session_dataset))


236


In [8]:
train_session = session_dataset[:-15]
valid_session = session_dataset[-15:]
len(train_session), len(valid_session)

(221, 15)

In [9]:
train_session[0]
# 첫번째 데이터
# context: 안녕하세요. 저는 고양이 6마리 키워요.
# response: 고양이를 6마리나요? 키우는거 안 힘드세요?

# 두번째 데이터
# context: 안녕하세요. 저는 고양이 6마리 키워요. | 고양이를 6마리나요? 키우는거 안 힘드세요?
# response: 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.

# 세번째 데이터
# context: 안녕하세요. 저는 고양이 6마리 키워요. | 고양이를 6마리나요? 키우는거 안 힘드세요? | 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.
# response: 가장 나이가 많은 고양이가 어떻게 돼요?

['안녕하세요. 저는 고양이 6마리 키워요.',
 '고양이를 6마리나요? 키우는거 안 힘드세요?',
 '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.',
 '가장 나이가 많은 고양이가 어떻게 돼요?',
 '여섯 살입니다. 갈색 고양이에요.',
 '그럼 가장 어린 고양이가 어떻게 돼요?',
 '한 살입니다. 작년에 분양 받았어요.',
 '그럼 고양이들끼리 안 싸우나요?',
 '저희 일곱은 다같이 한 가족입니다. 싸우는 일은 없어요.']

## positive / negative pairs 구성

In [10]:
from collections import defaultdict
train_json = defaultdict(dict)
count = 0
for session in train_session:
    context = [session[0]]
    for turn in range(1, len(session)):
        utt = session[turn] 
        train_json[count]['context'] = context[:]
        train_json[count]['response'] = utt
        train_json[count]['label'] = 1
        context.append(utt)
        count += 1
    break

In [11]:
train_json

defaultdict(dict,
            {0: {'context': ['안녕하세요. 저는 고양이 6마리 키워요.'],
              'response': '고양이를 6마리나요? 키우는거 안 힘드세요?',
              'label': 1},
             1: {'context': ['안녕하세요. 저는 고양이 6마리 키워요.',
               '고양이를 6마리나요? 키우는거 안 힘드세요?'],
              'response': '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.',
              'label': 1},
             2: {'context': ['안녕하세요. 저는 고양이 6마리 키워요.',
               '고양이를 6마리나요? 키우는거 안 힘드세요?',
               '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.'],
              'response': '가장 나이가 많은 고양이가 어떻게 돼요?',
              'label': 1},
             3: {'context': ['안녕하세요. 저는 고양이 6마리 키워요.',
               '고양이를 6마리나요? 키우는거 안 힘드세요?',
               '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.',
               '가장 나이가 많은 고양이가 어떻게 돼요?'],
              'response': '여섯 살입니다. 갈색 고양이에요.',
              'label': 1},
             4: {'context': ['안녕하세요. 저는 고양이 6마리 키워요.',
               '고양이를 6마리나요? 키우는거 안 힘드세요?',
               '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.',
               '가장 나이가 많은 고양이가 어떻

In [12]:
import random
use_turns = 5 # 꼭 5일 필요는 없음
neg_nums = 4 # 꼭 4일 필요는 없음

from collections import defaultdict
train_json = defaultdict(dict)
count = 0
for session in train_session:
    context = [session[0]]
    for turn in range(1, len(session)):
        utt = session[turn]
        train_json[count]['context'] = context[:][-use_turns:]
        train_json[count]['positive_response'] = utt
        context.append(utt)

        negative_candidates = random.sample(all_utts, neg_nums)
        train_json[count]['negative_responses'] = negative_candidates
        count += 1
    break

In [13]:
train_json

defaultdict(dict,
            {0: {'context': ['안녕하세요. 저는 고양이 6마리 키워요.'],
              'positive_response': '고양이를 6마리나요? 키우는거 안 힘드세요?',
              'negative_responses': ['부모님이랑 같이 창고 세일을 가기로 했습니다.',
               '종소리가 들리는 것을 보니 이제 곧 2부가 시작할 것 같아요.',
               '안녕하세요.',
               '로키 산맥은 만년설이 녹은 물이라서 강이 깨끗하기로 유명해요.']},
             1: {'context': ['안녕하세요. 저는 고양이 6마리 키워요.',
               '고양이를 6마리나요? 키우는거 안 힘드세요?'],
              'positive_response': '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.',
              'negative_responses': ['비단 인도 음식의 특징이 아니란 말씀이시네요.',
               '봄은 가을이랑 날씨가 비슷하지 않나요?',
               '좋아 보이는 곳이네요, 혹시 나중에 어디있는지 알려주실 수 있으신가요?',
               '스포일러는 하지 말아주세요, 전 아직 중간까지밖에 못 봤어요.']},
             2: {'context': ['안녕하세요. 저는 고양이 6마리 키워요.',
               '고양이를 6마리나요? 키우는거 안 힘드세요?',
               '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.'],
              'positive_response': '가장 나이가 많은 고양이가 어떻게 돼요?',
              'negative_responses': ['잘 지내지 못합니다. 오늘 의자가 부러졌어요.',
        

In [14]:
# import random
# neg_nums = 4

# from collections import defaultdict
# train_json = defaultdict(dict)
# count = 0
# for session in train_session:
#     context = [session[0]]
#     for turn in range(1, len(session)):
#         utt = session[turn]
#         train_json[count]['context'] = context[:][-use_turns:]
#         train_json[count]['positive_response'] = utt        
#         context.append(utt)        
        
#         negative_candidates = random.sample(all_utts, neg_nums)
#         train_json[count]['negative_responses'] = negative_candidates
#         count += 1
    
# import json
# with open('/content/drive/MyDrive/인공지능/멀티턴응답선택/korean_smile_style_dataset/train.json', 'w') as outfile:
#     json.dump(train_json, outfile)

In [15]:
import json, pdb
from torch.utils.data import Dataset
from transformers import AutoTokenizer

class fine_loader(Dataset):
    def __init__(self, data_path):
        self.tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')
        special_tokens = {'sep_token': '<SEP>'}
        self.tokenizer.add_special_tokens(special_tokens)
        
        """ 세션 데이터 """
        with open(data_path, 'r') as f:
            self.session_dataset = json.load(f)
        
    def __len__(self): # 기본적인 구성
        return len(self.session_dataset)
    
    def __getitem__(self, idx): # 기본적인 구성
        session = self.session_dataset[str(idx)]
        context = session['context']
        positive_response = session['positive_response']
        negative_responses = session['negative_responses']
        session_tokens = []
        session_labels = []
        
        """ MRS 입력 """
        context_token = [self.tokenizer.cls_token_id]
        for utt in context:
            context_token += self.tokenizer.encode(utt, add_special_tokens=False)
            context_token += [self.tokenizer.sep_token_id]
        
        pos_respons_token = [self.tokenizer.eos_token_id]
        pos_respons_token += self.tokenizer.encode(positive_response, add_special_tokens=False)
        positive_tokens = context_token + pos_respons_token
        session_tokens.append(positive_tokens)
        session_labels.append(1)
        
        for negative_response in negative_responses:
            neg_respons_token = [self.tokenizer.eos_token_id]
            neg_respons_token += self.tokenizer.encode(negative_response, add_special_tokens=False)
            negative_tokens = context_token + neg_respons_token        
            session_tokens.append(negative_tokens)
            session_labels.append(0)
        
        return session_tokens, session_labels

In [16]:
from torch.utils.data import DataLoader
train_path = '/content/drive/MyDrive/인공지능/멀티턴응답선택/korean_smile_style_dataset/train.json'
train_dataset = fine_loader(train_path)

In [17]:
session_tokens, session_labels = train_dataset[1]

In [18]:
len(session_tokens), len(session_tokens[0]), len(session_tokens[1])

(5, 44, 39)

In [19]:
train_dataset.tokenizer.decode(session_tokens[4])

'[CLS] 안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> [SEP] 저는 항상 옷을 같은 곳에서 삽니다. 직접 만들지는 않고요.'

In [20]:
session_labels

[1, 0, 0, 0, 0]

## 배치 처리하기

In [21]:
import json, pdb
from torch.utils.data import Dataset
from transformers import AutoTokenizer
import torch

class fine_loader(Dataset):
    def __init__(self, data_path):
        self.tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')
        special_tokens = {'sep_token': '<SEP>'}
        self.tokenizer.add_special_tokens(special_tokens)
        
        """ 세션 데이터 """
        with open(data_path, 'r') as f:
            self.session_dataset = json.load(f)
        
    def __len__(self): # 기본적인 구성
        return len(self.session_dataset)
    
    def __getitem__(self, idx): # 기본적인 구성
        session = self.session_dataset[str(idx)]
        context = session['context']
        positive_response = session['positive_response']
        negative_responses = session['negative_responses']
        session_tokens = []
        session_labels = []
        
        """ MRS 입력 """
        context_token = [self.tokenizer.cls_token_id]
        for utt in context:
            context_token += self.tokenizer.encode(utt, add_special_tokens=False)
            context_token += [self.tokenizer.sep_token_id]
        
        pos_respons_token = [self.tokenizer.eos_token_id]
        pos_respons_token += self.tokenizer.encode(positive_response, add_special_tokens=False)
        positive_tokens = context_token + pos_respons_token
        session_tokens.append(positive_tokens)
        session_labels.append(1)
        
        for negative_response in negative_responses:
            neg_respons_token = [self.tokenizer.eos_token_id]
            neg_respons_token += self.tokenizer.encode(negative_response, add_special_tokens=False)
            negative_tokens = context_token + neg_respons_token        
            session_tokens.append(negative_tokens)
            session_labels.append(0)
        
        return session_tokens, session_labels
    
    def collate_fn(self, sessions): # 배치를 위한 구성
        '''
            input:
                data: [(session1), (session2), ... ]
            return:
                batch_input_tokens_pad: (B, L) padded
                batch_labels: (B)
        '''        
        # 최대 길이 찾기 for padding
        max_input_len = 0
        for session in sessions:
            session_tokens, session_labels = session
            input_tokens_len = [len(x) for x in session_tokens]
            max_input_len = max(max_input_len, max(input_tokens_len))
        
        batch_input_tokens, batch_input_labels = [], []
        batch_input_attentions = []
        for session in sessions:
            session_tokens, session_labels = session
            for session_token in session_tokens:
                input_token = session_token + [self.tokenizer.pad_token_id for _ in range(max_input_len-len(session_token))]
                input_attention = [1 for _ in range(len(session_token))] + [0 for _ in range(max_input_len-len(session_token))]
                batch_input_tokens.append(input_token)
                batch_input_attentions.append(input_attention)
            batch_input_labels += session_labels
        
        return torch.tensor(batch_input_tokens), torch.tensor(batch_input_attentions), torch.tensor(batch_input_labels)

In [22]:
!touch dataset.py

In [23]:
from dataset import fine_loader
from torch.utils.data import DataLoader
train_path = '/content/drive/MyDrive/인공지능/멀티턴응답선택/korean_smile_style_dataset/train.json'
train_dataset = fine_loader(train_path)
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=False, num_workers=4, collate_fn=train_dataset.collate_fn)

In [24]:
for data in train_dataloader:
    batch_input_tokens, batch_input_attentions, batch_input_labels = data
    break

In [25]:
batch_input_tokens.shape, batch_input_attentions.shape, batch_input_labels.shape

(torch.Size([10, 53]), torch.Size([10, 53]), torch.Size([10]))

In [26]:
batch_input_tokens

tensor([[    0,  5891,  2205,  5971,    18,  1535,  2259,  7003,    26, 12736,
          5972,  2182,    18, 32000,     2,  7003,  2138,    26, 12736,  2075,
          2182,    35,  5688,  2259,  2180,  1378, 18941,  5971,    35,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1],
        [    0,  5891,  2205,  5971,    18,  1535,  2259,  7003,    26, 12736,
          5972,  2182,    18, 32000,     2, 10073, 22087,  2570,  2079,  5306,
          2256,  2955,  2069,  4920,  5723, 11800,    18,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1],
        [    0,  5891,  2205,  5971,    18,  1535,  2259,  7003,    26, 12736,
          5972,  2182,    18, 32000,     2,  4177,  4241,  2052,  597

## 모델링하기

In [27]:
from transformers import RobertaForMaskedLM, AutoTokenizer
import torch
import torch.nn as nn

class FineModel(nn.Module):
    def __init__(self):
        super(FineModel, self).__init__()
        self.model = RobertaForMaskedLM.from_pretrained('klue/roberta-base')
        self.hiddenDim = self.model.config.hidden_size
        
        self.tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')
        special_tokens = {'sep_token': '<SEP>'}
        self.tokenizer.add_special_tokens(special_tokens)
        self.model.resize_token_embeddings(len(self.tokenizer))        
        
        """ score matrix """
        # self.W = nn.Linear(self.hiddenDim, 3)
        self.W2 = nn.Linear(self.hiddenDim, 2)
        
    def forward(self, batch_input_tokens, batch_input_attentions):
        ## Binary Classification (pointwise)
        outputs = self.model(batch_input_tokens, attention_mask=batch_input_attentions, output_hidden_states=True)['hidden_states'][-1] # [B, L, hidden_dim]
        cls_outputs = outputs[:,0,:] # [B, hidden_dim]/0은 cls를 해당
        cls_logits = self.W2(cls_outputs) # [B, 2]
         
        return cls_logits

In [28]:
!touch model.py

In [29]:
import torch
from model import FineModel
fine_model = FineModel()
result = fine_model(batch_input_tokens, batch_input_attentions)

In [30]:
result

tensor([[ 0.0708, -0.1702],
        [ 0.0855, -0.1513],
        [ 0.0904, -0.1460],
        [ 0.0990, -0.1356],
        [ 0.0804, -0.1734],
        [ 0.0823, -0.1648],
        [ 0.0714, -0.1668],
        [ 0.0550, -0.1730],
        [ 0.0878, -0.1675],
        [ 0.0696, -0.1312]], grad_fn=<AddmmBackward0>)

In [31]:
fine_model.load_state_dict(torch.load('/content/drive/MyDrive/인공지능/멀티턴응답선택/post_model.bin'))
result2 = fine_model(batch_input_tokens, batch_input_attentions)


RuntimeError: ignored

In [32]:
# key가 안맞는 부분은 초기화 사용
# fine_model = FineModel()
fine_model.load_state_dict(torch.load('/content/drive/MyDrive/인공지능/멀티턴응답선택/post_model.bin'), strict=False)

_IncompatibleKeys(missing_keys=['W2.weight', 'W2.bias'], unexpected_keys=['W.weight', 'W.bias'])

In [33]:
result2 = fine_model(batch_input_tokens, batch_input_attentions)
result2

tensor([[ 0.1175, -0.0613],
        [ 0.3246, -0.0514],
        [ 0.4504,  0.3256],
        [ 0.3470, -0.0222],
        [ 0.3598,  0.4286],
        [ 0.0173, -0.0816],
        [ 0.2144,  0.0394],
        [ 0.2717,  0.0225],
        [ 0.4633,  0.1211],
        [ 0.4383,  0.1879]], grad_fn=<AddmmBackward0>)